# DST Airlines

[LINK](https://docs.google.com/document/d/1wyo5QOe8UZZE0_fm0PNdc_bTUNqjyrBMHSHwH-JfSEM/edit)

Description détaillée :  
De nos jours, il est possible d’avoir des informations sur les vols dans le monde entier et de traquer en temps réel un avion. Nous pouvons observer ce site en guise d’exemple.  
Le but ici est de s’en approcher le plus possible en passant par des API de différentes compagnies aériennes.

Etape | Description | Modules / Masterclass / Templates | Objectif | Conditions de validation du projet
----- | ----------- | --------------------------------- | -------- | ---------------------------------
1 | Exploration de données non structurées | Passer par l’API de Lufthansa pour récupérer des données sur les vols | 103 - Introduction aux APIs de données | Un fichier explicatif du traitement et des différentes données accessibles (doc / pdf). Un exemple de données collectées.


[Lufthansa Open API Documentation](https://developer.lufthansa.com/docs)

[LH OpenAPI testing](https://developer.lufthansa.com/io-docs)

[Public API Method Details](https://developer.lufthansa.com/docs/read/api_details)